In [1]:
import scripts.data_pull as dp
import scripts.data_store as ds
import scripts.bias_assignment as bias
import pandas as pd

https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

In [2]:
# Reading Raw Data into Pandas Dataframe from Pickle
jacobin_text = dp.data_pull("jacobin_text")
brooking_text = dp.data_pull("brooking_text")
heritage_com_text = dp.data_pull("heritage_com_text")
heritage_rep_text = dp.data_pull("heritage_rep_text")
american_mind_features_text = dp.data_pull("american_mind_features_text")
american_mind_memos_text = dp.data_pull("american_mind_memos_text")
american_mind_salvos_text = dp.data_pull("american_mind_salvos_text")

In [3]:
data_corpus = pd.concat([jacobin_text, brooking_text, heritage_com_text, heritage_rep_text, american_mind_features_text, american_mind_memos_text, american_mind_salvos_text])

In [4]:
data_corpus = data_corpus.dropna()

In [5]:
corpus_type_dict = {'text': 'string', 'art_title': 'string', 'art_author': 'string', 'art_topic': 'string', 'art_link': 'string', 'art_source': 'string'}
corpus_cols = ['text', 'art_title', 'art_author', 'art_date', 'art_topic', 'art_link', 'art_source']

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html

In [6]:
data_corpus = data_corpus.astype(corpus_type_dict)
data_corpus['art_date'] = pd.to_datetime(data_corpus['art_date'], yearfirst= True)

In [7]:
data_corpus.dtypes

text                  string
art_title             string
art_author            string
art_date      datetime64[ns]
art_topic             string
art_link              string
art_source            string
dtype: object

https://stackoverflow.com/questions/36392735/how-to-combine-multiple-rows-into-a-single-row-with-pandas

In [8]:
full_corpus = data_corpus.groupby(corpus_cols[1:])['text'].apply(' '.join).reset_index()

In [9]:
full_corpus['text'] = full_corpus['text'].astype(str).str.lower()
full_corpus['art_topic'] = full_corpus['art_topic'].str.lower()
full_corpus['art_bias'] = full_corpus["art_source"].apply(bias.bias_assignment)

In [10]:
full_corpus.head()

,art_title,art_author,art_date,art_topic,art_link,art_source,text,art_bias
0,The emerging markets e-commerce opportunity,Christoph Ungerer,2021-03-26,future-development,https://www.brookings.edu/blog/future-developm...,Brookings Institute,while e-commerce giants such as amazon dominat...,left-wing
1,"""1619"" Pulitzer Will Boost Socialist Teaching ...",Mike Gonzalez,2020-05-11,education,https://www.heritage.org/education/commentary/...,Heritage Commentary,the pulitzer prize board this week awarded its...,right-wing
2,"""1983"" and ""The Brink"" Review: The Most Danger...","Lee Edwards, Ph.D.",2018-08-10,defense,https://www.heritage.org/defense/commentary/19...,Heritage Commentary,most historians agree that the world came clos...,right-wing
3,"""60 Minutes"" Snubs the Facts on Education",Jonathan Butcher,2018-03-13,education,https://www.heritage.org/education/commentary/...,Heritage Commentary,beth richardson is committed to her son’s succ...,right-wing
4,"""As Israel and the Arabs Battle, Moscow Collec...",James Phillips,1983-09-20,europe,https://www.heritage.org/europe/report/israel-...,Heritage Report,"i i 291 september 20, 1983 as israel and the a...",right-wing


In [12]:
ds.data_store(full_corpus, data_filename= 'combined_corpus')